In [ ]:
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
from tqdm import tqdm
import re
import pandas as pd
import json

In [ ]:
youtube_api_key = json.load(open('secrets.json'))['youtube_api_key']
melon_soup = bs(open("tnd").read(),'html.parser')
video_ids = [vid['href'].replace('/watch?v=','') for vid in melon_soup.findAll('a', {'class':'yt-simple-endpoint style-scope ytd-grid-video-renderer'}, href=True)]
melon_reviews_raw = [json.loads(urlopen(f"https://youtube.googleapis.com/youtube/v3/videos?part=snippet%2CcontentDetails%2Cstatistics%2CtopicDetails%2Cid&id={id}&key={youtube_api_key}").read()) for id in tqdm(video_ids, desc='mining_reviews')]

In [ ]:
melon_reviews = []
for rev in melon_reviews_raw:
    title = rev['items'][0]['snippet']['title']
    if 'album review' in title.lower():

        orig_desc = rev['items'][0]['snippet']['description']
        clean_desc = re.sub(r'http\S+', '<URL>', orig_desc).replace('==','').replace('\n',' ').lower()

        split_title = title.split('-')
        if len(split_title)>=2:
            artist,album = split_title[0].strip(),split_title[1].strip()
        else:
            artist,album = split_title[0],split_title[0]

        score = re.compile(r'[0-9]+\/10').search(orig_desc)
        if score:
            score = int(score.group(0).split('/')[0])
            if score>10 or score<0: score=None

        genres = [p for p in orig_desc.split('\n\n') if ' / ' in p]
        if genres: genres = genres[0].split(' / ')[-1].lower().split(', ')
        else: genres = None

        melon_reviews.append({
            'id':rev['items'][0]['id'],
            'artist':artist,
            'album':album.replace(' ALBUM REVIEW',''),
            'date':rev['items'][0]['snippet']['publishedAt'],
            'duration':rev['items'][0]['contentDetails']['duration'],
            'description':clean_desc.lower(),
            'thumbnail':rev['items'][0]['snippet']['thumbnails']['high']['url'],
            'views':rev['items'][0]['statistics'].get('viewCount',None),
            'score':score,
            'genre':genres
        })

In [ ]:
melon_df = pd.DataFrame(melon_reviews)
melon_df